In [3]:
# ============================================
# CLIENTE INTEGRAL - RESIDENCIALES Y COMERCIALES (ACTUALIZADO)
# ============================================

import pandas as pd
import numpy as np
import random

np.random.seed(42)
random.seed(42)

# -----------------------------
# 1) Bases separadas
# -----------------------------
def crear_base_clientes(tipo, areas, n=50):
    clientes = [f"{tipo[:3].upper()}_{str(i).zfill(3)}" for i in range(1, n + 1)]
    data = {
        "id_cliente": clientes,
        "tipo_cliente": tipo.capitalize(),
    }
    # Genera una columna binaria (0/1) por cada área
    for area in areas:
        data[area] = np.random.randint(0, 2, n)

    df = pd.DataFrame(data)
    df.to_csv(f"base_clientes_{tipo}.csv", index=False)
    print(f"💾 Guardado: base_clientes_{tipo}.csv ({n} registros)")
    return df


areas_residencial = ["brilla", "consumo", "sad"]
areas_comercial = ["brilla", "consumo", "efisoluciones"]

df_residenciales = crear_base_clientes("residencial", areas_residencial)
df_comerciales   = crear_base_clientes("comercial", areas_comercial)


# -----------------------------
# 2) Función de generación de dimensiones
# -----------------------------
def generar_dimensiones_area(df_base, area, tipo, min_frac=0.6, max_frac=0.9):
    elegibles = df_base.loc[df_base[area] == 1, "id_cliente"].tolist()
    if not elegibles:
        return pd.DataFrame(columns=[
            "id_cliente", "dimension_economica", "dimension_relacional",
            "dimension_cumplimiento", "dimension_potencial"
        ])

    k_min = max(1, int(len(elegibles) * min_frac))
    k_max = max(k_min, int(len(elegibles) * max_frac))
    k = random.randint(k_min, k_max)
    subset = random.sample(elegibles, k)

    df_dim = pd.DataFrame({
        "id_cliente": subset,
        "dimension_economica": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
        "dimension_relacional": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
        "dimension_cumplimiento": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
        "dimension_potencial": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
        "tipo_cliente": tipo.capitalize(),
    })

    df_dim.to_csv(f"dimensiones_{area}_{tipo}.csv", index=False)
    print(f"💾 Guardado: dimensiones_{area}_{tipo}.csv "
          f"(clientes elegibles: {len(elegibles)}, usados: {len(subset)})")
    return df_dim


# -----------------------------
# 3) Crear dimensiones por área y tipo
# -----------------------------
# --- Residenciales ---
df_brilla_res = generar_dimensiones_area(df_residenciales, "brilla", "residencial")
df_consumo_res = generar_dimensiones_area(df_residenciales, "consumo", "residencial")
df_sad_res = generar_dimensiones_area(df_residenciales, "sad", "residencial")

# --- Comerciales ---
df_brilla_com = generar_dimensiones_area(df_comerciales, "brilla", "comercial")
df_consumo_com = generar_dimensiones_area(df_comerciales, "consumo", "comercial")
df_efisoluciones_com = generar_dimensiones_area(df_comerciales, "efisoluciones", "comercial")


# -----------------------------
# 4) Consolidado por tipo de cliente
# -----------------------------
df_dimensiones_residencial = pd.concat([
    df_brilla_res.assign(area="brilla"),
    df_consumo_res.assign(area="consumo"),
    df_sad_res.assign(area="sad"),
], ignore_index=True)

df_dimensiones_residencial.to_csv("dimensiones_todas_residencial.csv", index=False)
print("💾 Guardado: dimensiones_todas_residencial.csv")

df_dimensiones_comercial = pd.concat([
    df_brilla_com.assign(area="brilla"),
    df_consumo_com.assign(area="consumo"),
    df_efisoluciones_com.assign(area="efisoluciones"),
], ignore_index=True)

df_dimensiones_comercial.to_csv("dimensiones_todas_comercial.csv", index=False)
print("💾 Guardado: dimensiones_todas_comercial.csv")


# -----------------------------
# 5) Verificación de consistencia
# -----------------------------
def chequear_consistencia(df_base, df_dim, area, tipo):
    if df_dim.empty:
        print(f"🔎 {area}_{tipo}: sin filas (no había elegibles).")
        return
    merged = df_dim.merge(df_base[["id_cliente", area]], on="id_cliente", how="left")
    inconsistentes = merged.loc[merged[area] != 1]
    if inconsistentes.empty:
        print(f"✅ Consistencia {area}_{tipo}: OK (solo clientes con 1).")
    else:
        print(f"⚠️ Inconsistencia {area}_{tipo}: hay clientes con 0 en el área.")
        print(inconsistentes)

# Verificar residenciales
for area in areas_residencial:
    df_area = {"brilla": df_brilla_res, "consumo": df_consumo_res, "sad": df_sad_res}[area]
    chequear_consistencia(df_residenciales, df_area, area, "residencial")

# Verificar comerciales
for area in areas_comercial:
    df_area = {"brilla": df_brilla_com, "consumo": df_consumo_com, "efisoluciones": df_efisoluciones_com}[area]
    chequear_consistencia(df_comerciales, df_area, area, "comercial")

print("✅ Generación completa: bases y dimensiones separadas para residenciales y comerciales.")

💾 Guardado: base_clientes_residencial.csv (50 registros)
💾 Guardado: base_clientes_comercial.csv (50 registros)
💾 Guardado: dimensiones_brilla_residencial.csv (clientes elegibles: 27, usados: 17)
💾 Guardado: dimensiones_consumo_residencial.csv (clientes elegibles: 29, usados: 17)
💾 Guardado: dimensiones_sad_residencial.csv (clientes elegibles: 26, usados: 18)
💾 Guardado: dimensiones_brilla_comercial.csv (clientes elegibles: 18, usados: 14)
💾 Guardado: dimensiones_consumo_comercial.csv (clientes elegibles: 24, usados: 16)
💾 Guardado: dimensiones_efisoluciones_comercial.csv (clientes elegibles: 25, usados: 15)
💾 Guardado: dimensiones_todas_residencial.csv
💾 Guardado: dimensiones_todas_comercial.csv
✅ Consistencia brilla_residencial: OK (solo clientes con 1).
✅ Consistencia consumo_residencial: OK (solo clientes con 1).
✅ Consistencia sad_residencial: OK (solo clientes con 1).
✅ Consistencia brilla_comercial: OK (solo clientes con 1).
✅ Consistencia consumo_comercial: OK (solo clientes co